In [1]:
import pandas as pd
import sqlite3

In [2]:
orders = pd.read_csv(r"C:\Users\Purus\OneDrive\Desktop\orders.csv")

In [3]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         10000 non-null  int64  
 1   user_id          10000 non-null  int64  
 2   restaurant_id    10000 non-null  int64  
 3   order_date       10000 non-null  object 
 4   total_amount     10000 non-null  float64
 5   restaurant_name  10000 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 468.9+ KB


In [4]:
users = pd.read_json(r"C:\Users\Purus\OneDrive\Desktop\users.json")

In [5]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     3000 non-null   int64 
 1   name        3000 non-null   object
 2   city        3000 non-null   object
 3   membership  3000 non-null   object
dtypes: int64(1), object(3)
memory usage: 93.9+ KB


In [6]:
conn = sqlite3.connect(":memory:")

In [7]:
with open(r"C:\Users\Purus\OneDrive\Desktop\restaurants.sql", "r") as file:
    sql_script = file.read()
conn.executescript(sql_script)


In [8]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

In [9]:
restaurants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   restaurant_id    500 non-null    int64  
 1   restaurant_name  500 non-null    object 
 2   cuisine          500 non-null    object 
 3   rating           500 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 15.8+ KB


In [10]:
df = orders.merge(users,on="user_id",how="left")

In [11]:
df["city"].isna().sum()

np.int64(0)

In [12]:
df = df.merge(restaurants,on="restaurant_id",how="left",suffixes=("_order2", "_master2"))
df.drop(columns=["restaurant_name_order2"], inplace=True)
df.rename(columns={"restaurant_name_master2": "restaurant_name"}, inplace=True)


In [13]:
df["order_date"] = pd.to_datetime(df["order_date"])

C:\Users\lovel\AppData\Local\Temp\ipykernel_14868\4261617660.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"])


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   order_id         10000 non-null  int64         
 1   user_id          10000 non-null  int64         
 2   restaurant_id    10000 non-null  int64         
 3   order_date       10000 non-null  datetime64[ns]
 4   total_amount     10000 non-null  float64       
 5   name             10000 non-null  object        
 6   city             10000 non-null  object        
 7   membership       10000 non-null  object        
 8   restaurant_name  10000 non-null  object        
 9   cuisine          10000 non-null  object        
 10  rating           10000 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(5)
memory usage: 859.5+ KB


In [15]:
df["year"] = df["order_date"].dt.year
df["month"] = df["order_date"].dt.month
df["quarter"] = df["order_date"].dt.to_period("Q")

In [16]:
print(df[df["membership"]=="Gold"].groupby("city")["total_amount"].sum().idxmax())

Chennai


In [17]:
print(df.groupby("cuisine")["total_amount"].mean().idxmax())

Mexican


In [18]:
cnt = df.groupby("user_id")["total_amount"].sum().gt(1000).sum()
print(cnt)

2544


In [19]:
df["rating_range"] = pd.cut(df["rating"], [3.0,3.5,4.0,4.5,5.0])
print(df.groupby("rating_range")["total_amount"].sum().idxmax())

(4.5, 5.0]


C:\Users\lovel\AppData\Local\Temp\ipykernel_14868\3267684469.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby("rating_range")["total_amount"].sum().idxmax())


In [20]:
print(df[df["membership"]=="Gold"].groupby("city")["total_amount"].mean().idxmax())

Chennai


In [21]:
summary = pd.DataFrame({
    "restaurants": df.groupby("cuisine")["restaurant_id"].nunique(),
    "revenue": df.groupby("cuisine")["total_amount"].sum()
})


In [22]:
print(summary.sort_values(["restaurants","revenue"], ascending=[True,False]).head(1))

         restaurants     revenue
cuisine                         
Chinese          120  1930504.65


In [23]:
print(round((df[df["membership"]=="Gold"].shape[0]/df.shape[0])*100), "%")

50 %


In [24]:
rs = df.groupby("restaurant_name").agg(
    avg=("total_amount","mean"),
    cnt=("order_id","count")
)
print(rs[rs["cnt"]<20].sort_values("avg",ascending=False).head(1))


                         avg  cnt
restaurant_name                  
Restaurant_294   1040.222308   13


In [25]:
restaurants[restaurants["restaurant_id"] == 294]

,restaurant_id,restaurant_name,cuisine,rating
293,294,Restaurant_294,Italian,3.1


In [26]:
print(df.groupby(["membership","cuisine"])["total_amount"].sum().idxmax())

('Regular', 'Mexican')


In [27]:
print(df.groupby("quarter")["total_amount"].sum().idxmax())


2023Q3


In [28]:
df["order_date"] = pd.to_datetime(df["order_date"])

In [29]:
gold_orders = df[df["membership"]=="Gold"].shape[0]
gold_orders


4987

In [30]:
hyd_revenue = round(df[df["city"]=="Hyderabad"]["total_amount"].sum())
hyd_revenue

1889367

In [31]:
distinct_users = df["user_id"].nunique()
distinct_users


2883

In [32]:
gold_aov = round(df[df["membership"]=="Gold"]["total_amount"].mean(), 2)
gold_aov


np.float64(797.15)

In [33]:
high_rating_orders = df[df["rating"] >= 4.5].shape[0]
high_rating_orders

3374

In [34]:
top_gold_city = (
    df[df["membership"]=="Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)
orders_top_city_gold = df[
    (df["membership"]=="Gold") & (df["city"]==top_gold_city)
].shape[0]

orders_top_city_gold



1337